In [75]:
from hyppo.ksample import KSample
from hyppo.independence import Dcorr
from combat import combat
import pandas as pd
import glob
import os
import graspy as gp
import numpy as np
from dask.distributed import Client, progress
import dask.dataframe as ddf
from scipy.stats import zscore, rankdata, mannwhitneyu
import copy
import math
import networkx as nx
from graspy.models import SIEMEstimator as siem
import re

In [76]:
def get_sub_pheno_dat(subid, scan, pheno_dat):
    matches = pheno_dat.index[pheno_dat["SUBID"] == int(subid)].tolist()
    match = np.min(matches)
    return(int(pheno_dat.iloc[match]["SEX"]))

def get_age_pheno_dat(subid, scan, pheno_dat):
    matches = pheno_dat.index[pheno_dat["SUBID"] == int(subid)].tolist()
    match = np.min(matches)
    return(float(pheno_dat.iloc[match]["AGE_AT_SCAN_1"]))

def apply_along_dataset(scs, dsets, fn):
    scs_xfmd = np.zeros(scs.shape)
    for dset in np.unique(dsets):
        scs_xfmd[dsets == dset,:] = np.apply_along_axis(fn, 0, scs[dsets == dset,:])
    return(scs_xfmd)

def apply_along_individual(scs, fn):
    scs_xfmd = np.zeros(scs.shape)

def zsc(x):
    x_ch = copy.deepcopy(x)
    if (np.var(x_ch) > 0):
        x_ch = (x_ch - np.mean(x_ch))/np.std(x_ch)
        return x_ch
    else:
        return np.zeros(x_ch.shape)

def ptr(x):
    x_ch = copy.deepcopy(x)
    nz = x[x != 0]
    x_rank = rankdata(nz)*2/(len(nz) + 1)
    x_ch[x_ch != 0] = x_rank
    if (np.min(x_ch) != np.max(x_ch)):
        x_ch = (x_ch - np.min(x_ch))/(np.max(x_ch) - np.min(x_ch))
    return(x_ch)

In [79]:
# path to directory produced by download_aws.sh
basepath = '/mnt/nfs2/MR/corr/corr_m2g/graphs/m2g/fmri/'

# path to directory containing phenotypic annotations for download_aws.sh script
pheno_basepath = '/mnt/nfs2/MR/corr/corr_m2g/phenotypic/CoRR_AggregatedPhenotypicData.csv'
pheno_dat = pd.read_csv(pheno_basepath)
datasets = os.listdir(basepath)
print(datasets)

['IBATRT', 'Utah1', 'BMB_1', 'IPCAS_2', 'ABIDEII-TCD_1', 'SWU1', 'UWM', 'BNU2', 'XHCUMS', 'SWU4', 'IPCAS_3', 'ABIDEII-SDSU_1', 'SWU3', 'IPCAS_4', 'NYU_2', 'IPCAS_1', 'IPCAS_7', 'UPSM_1', 'ABIDEII-BNI_1', 'IACAS_1', 'IPCAS_5', 'NYU_1', 'BNU1', 'MRN_1', 'BNU3', 'HNU1', 'SWU2', 'IPCAS_8', 'JHNU', 'IPCAS_6']


In [80]:
fmri_dict = {}

for i, dataset in enumerate(datasets):
    try:
        dset_dir = os.path.join('{}{}'.format(basepath, dataset), '*.csv')
        files_ds = glob.glob(dset_dir)

        successes = len(files_ds)

        scans = []
        sexs = []
        ages = []
        ds_lab = []
        subjects = []
        subids = []
        sessions = []
        for f in files_ds:
            # obtain graph for this subject
            try:
                gr_dat = gp.utils.import_edgelist(f).flatten()
                scansub = re.split('-|_', os.path.basename(f))
                sex = get_sub_pheno_dat(scansub[1], scansub[3], pheno_dat)
                age = get_age_pheno_dat(scansub[1], scansub[3], pheno_dat)
                subid = "dataset-{}_sub-{}_ses-{}".format(dataset, scansub[1], scansub[3])
                scans.append(gr_dat)
                sexs.append(sex)
                ages.append(age)
                subjects.append(scansub[1])
                ds_lab.append(dataset)
                subids.append(subid)
                sessions.append(scansub[3])
            except Exception as e:
                successes -= 1

        if (successes < 5):
            raise ValueError("Dataset: {} does not have enough successes.".format(dataset))

        # add it in assuming there are enough unique files with metadata annotation
        scans = np.vstack(scans)
        fmri_dict[dataset] = {"Data": scans, "Subject": subjects, "Session": sessions, "Subid": subids,
                              "Sex": sexs, "Age": ages, "Dataset": ds_lab}

    except Exception as e:
        print("Error in {} Dataset.".format(dataset))
        print(e)

Error in ABIDEII-TCD_1 Dataset.
Dataset: ABIDEII-TCD_1 does not have enough successes.
Error in ABIDEII-SDSU_1 Dataset.
Dataset: ABIDEII-SDSU_1 does not have enough successes.
Error in ABIDEII-BNI_1 Dataset.
Dataset: ABIDEII-BNI_1 does not have enough successes.


In [81]:
ncores = 99
client = Client(threads_per_worker=1, n_workers=ncores)

/home/eric/.virtualenvs/batch/lib/python3.6/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 32855 instead
  http_address["port"], self.http_server.port


## Preservation of Network Statistics

In [82]:
def diag_edges(n):
    """
    A function for generating diagonal SIEM edge communities.
    """
    m = int(n/2)
    edge_comm = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if (i == j + m) or (j == i + m):
                edge_comm[i,j] = 1
            else:
                edge_comm[i,j] = 2
    np.fill_diagonal(edge_comm, 0)
    return edge_comm

def modular_edges(n):
    """
    A function for generating modular sbm edge communities.
    """
    m = int(n/2)
    edge_comm = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if ((i<m) & (j<m)) or ( (i>=m ) & (j>=m) ):
                edge_comm[i,j] = 1
            else:
                edge_comm[i,j] = 2
    np.fill_diagonal(edge_comm, 0)
    return edge_comm

des_diag = diag_edges(70)
des_mod = modular_edges(70)

def mww(G, C):
    A = G[C == 1]
    B = G[C == 2]
    test_res = list(mannwhitneyu(A, B, alternative='greater'))
    test_res.append(np.mean(A))
    test_res.append(np.mean(B))
    return(test_res)

In [83]:
dset_ls = [fmri_dict[ds]["Data"] for ds in fmri_dict.keys()]
raw_dat = np.vstack(dset_ls)
datasets = np.array([j for ds in fmri_dict.keys() for j in fmri_dict[ds]["Dataset"]])
# get the subject ids and dataset ids as a big list
subjects = np.array([j for ds in fmri_dict.keys() for j in fmri_dict[ds]["Subject"]])
sessions = np.array([j for ds in fmri_dict.keys() for j in fmri_dict[ds]["Session"]])
subids = np.array([j for ds in fmri_dict.keys() for j in fmri_dict[ds]["Subid"]])
sexs = np.array([j for ds in fmri_dict.keys() for j in fmri_dict[ds]["Sex"]])
ages = np.array([j for ds in fmri_dict.keys() for j in fmri_dict[ds]["Age"]])

In [84]:
raw_dat.shape

(3545, 4900)

In [85]:
def prepare_aggregate_data(scans, datasets):
    newdat = {}
    newdat["raw"] = copy.deepcopy(scans)
    # copy the raw data over
    newdat["zscore"] = copy.deepcopy(scans)
    newdat["ptr"] = copy.deepcopy(scans)
    newdat["combat"] = copy.deepcopy(scans)

    # remove stationary edges for combat
    combat_rem_edges = ~np.all(newdat["combat"] == 0, axis=0)

    # apply relevant transforms en-masse
    newdat["zscore"] = apply_along_dataset(newdat["zscore"], datasets, zscore)
    # replace nans with zeros
    newdat["zscore"][np.isnan(newdat["zscore"])] = 0
    newdat["ptr"] = apply_along_dataset(newdat["ptr"], datasets, ptr)
    newdat["combat"][:,combat_rem_edges] = np.array(combat(pd.DataFrame(newdat["combat"][:,combat_rem_edges].T), datasets, model=None, numerical_covariates=None)).T
    return(newdat)

data_preproc = {}
data_preproc["raw"] = prepare_aggregate_data(raw_dat, datasets)
data_preproc["ptr"] = prepare_aggregate_data(np.apply_along_axis(ptr, 1, raw_dat), datasets)

found 27 batches
found 0 numerical covariates...
found 0 categorical variables:	
Standardizing Data across genes.
Fitting L/S model and finding priors
Finding parametric adjustments


Adjusting data


found 27 batches
found 0 numerical covariates...
found 0 categorical variables:	
Standardizing Data across genes.
Fitting L/S model and finding priors
Finding parametric adjustments


Adjusting data


In [86]:
exps = []

for i, sub in enumerate(subjects):
    for sxfm in ["raw", "ptr"]:
        for dxfm in ["raw", "zscore", "ptr", "combat"]:
            exps.append([datasets[i], subjects[i], sessions[i], sexs[i], ages[i], i, sub, sxfm, dxfm])
sim_exps = pd.DataFrame(exps, columns=["Dataset", "Subject", "Retest", "Sex", "Age",
                                       "Ix", "Fullname", "Sxfm", "Dxfm"])
print(sim_exps.head(n=20))

   Dataset  Subject Retest  Sex   Age  Ix Fullname Sxfm    Dxfm
0   IBATRT  0027241     11    2  27.0   0  0027241  raw     raw
1   IBATRT  0027241     11    2  27.0   0  0027241  raw  zscore
2   IBATRT  0027241     11    2  27.0   0  0027241  raw     ptr
3   IBATRT  0027241     11    2  27.0   0  0027241  raw  combat
4   IBATRT  0027241     11    2  27.0   0  0027241  ptr     raw
5   IBATRT  0027241     11    2  27.0   0  0027241  ptr  zscore
6   IBATRT  0027241     11    2  27.0   0  0027241  ptr     ptr
7   IBATRT  0027241     11    2  27.0   0  0027241  ptr  combat
8   IBATRT  0027240     22    2  40.0   1  0027240  raw     raw
9   IBATRT  0027240     22    2  40.0   1  0027240  raw  zscore
10  IBATRT  0027240     22    2  40.0   1  0027240  raw     ptr
11  IBATRT  0027240     22    2  40.0   1  0027240  raw  combat
12  IBATRT  0027240     22    2  40.0   1  0027240  ptr     raw
13  IBATRT  0027240     22    2  40.0   1  0027240  ptr  zscore
14  IBATRT  0027240     22    2  40.0   

In [87]:
def singlegraph_exp(row):
    # grab data, and reshape it to nv x nv matrix
    flat_gr = data_preproc[row[7]][row[8]][row[5],:]
    nv = int(np.sqrt(np.max(flat_gr.shape)))
    exp_gr = flat_gr.reshape((nv, nv))
    G = nx.from_numpy_matrix(exp_gr)
    cc = nx.average_clustering(G, weight="weight")
    deg = np.array(list(dict(G.degree(weight="weight")).values())).mean()
    homophilic = mww(exp_gr, des_mod)
    homotopic = mww(exp_gr, des_diag)
    return(row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7],
           row[8], cc, deg, homophilic[2], homotopic[2], homophilic[3], homotopic[3], 
           homophilic[1], homotopic[1], homophilic[0], homotopic[0])

In [88]:
sim_exps = ddf.from_pandas(sim_exps, npartitions=ncores)
sim_results = sim_exps.apply(lambda x: singlegraph_exp(x), axis=1, result_type='expand',
                             meta={0: str, 1: str, 2: str, 3:str, 4:str, 5:str, 6:str, 7:str, 8:str,
                                   9: float, 10: float, 11: float, 12: float, 13: float, 14: float,
                                  15: float, 16: float, 17: float, 18: float})
sim_results

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
npartitions=99,,,,,,,,,,,,,,,,,,,
0,object,object,object,object,object,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
287,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28126,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28359,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [89]:
sim_results = sim_results.compute(scheduler="multiprocessing")
sim_results = sim_results.rename(columns={0: "Dataset", 1: "Subject", 2: "Retest", 3: "Sex", 4: "Age", 5: "Ix",
                                          6: "Fullname", 7: "Sxfm", 8: "Dxfm", 9: "Clustering",
                                          10: "Degree", 11: "Homophilic_mean", 12: "Homotopic_mean",
                                          13: "Heterophilic_mean", 14: "Heterotopic_mean",
                                          15: "Homophilic_pvalue", 16: "Homotopic_pvalue",
                                          17: "Homophilic_stat", 18: "Homotopic_stat"})
sim_results.to_csv('../data/summary/batch_statistics.csv')
sim_results.head(n=30)

,Dataset,Subject,Retest,Sex,Age,Ix,Fullname,Sxfm,Dxfm,Clustering,Degree,Homophilic_mean,Homotopic_mean,Heterophilic_mean,Heterotopic_mean,Homophilic_pvalue,Homotopic_pvalue,Homophilic_stat,Homotopic_stat
0,IBATRT,0027241,11,2,27.0,0,0027241,raw,raw,0.412969+0.000000j,29.562112,0.454585,0.735618,0.403034,0.423919,8.963445e-20,2.565275e-27,3352780.0,291260.0
1,IBATRT,0027241,11,2,27.0,0,0027241,raw,zscore,0.210416+0.080516j,6.282878,0.203380,-0.386291,-0.018059,0.098076,7.399481e-20,9.989675e-01,3353796.0,130920.0
2,IBATRT,0027241,11,2,27.0,0,0027241,raw,ptr,0.480426+0.000000j,36.602165,0.564583,0.400866,0.497324,0.532372,4.358851e-19,9.999165e-01,3344282.0,123006.0
3,IBATRT,0027241,11,2,27.0,0,0027241,raw,combat,0.356183+0.002274j,25.488772,0.396074,0.671906,0.343493,0.364954,7.301799e-22,1.018622e-26,3377608.0,289780.0
4,IBATRT,0027241,11,2,27.0,0,0027241,ptr,raw,0.439415+0.000000j,34.510715,0.538199,0.868901,0.463199,0.494733,8.963445e-20,2.565275e-27,3352780.0,291260.0
5,IBATRT,0027241,11,2,27.0,0,0027241,ptr,zscore,0.231486+0.095148j,-1.820399,0.105771,-0.930452,-0.154760,-0.013087,5.152017e-20,9.997842e-01,3355708.0,125832.0
6,IBATRT,0027241,11,2,27.0,0,0027241,ptr,ptr,0.437185+0.000000j,34.460576,0.536755,0.365125,0.463168,0.501404,3.880999e-19,9.999464e-01,3344920.0,121736.0
7,IBATRT,0027241,11,2,27.0,0,0027241,ptr,combat,0.426333+0.001770j,34.359289,0.538814,0.839413,0.458275,0.492939,3.949446e-23,7.279788e-24,3392048.0,282472.0
8,IBATRT,0027240,22,2,40.0,1,0027240,raw,raw,0.330571+0.000000j,24.022427,0.353413,0.786680,0.343039,0.341702,9.105984e-03,2.259462e-43,3029906.0,325952.0
9,IBATRT,0027240,22,2,40.0,1,0027240,raw,zscore,0.189065+0.118694j,-26.740332,-0.399016,-0.058594,-0.376394,-0.392378,8.562283e-01,2.577096e-04,2863972.0,206820.0


## Save Example Connectome for each option type

In [114]:
refsub = "0025864"; refses = "1"
nv = 70

row_ix = np.zeros((nv, nv)) 
col_ix = np.zeros((nv, nv))
for i in range(70):
    for j in range(70):
        col_ix[i,j] = j
        row_ix[i,j] = i
row_ix = row_ix.flatten()
col_ix = col_ix.flatten()

data = []
data_avg = []
for sxfm, data_preproc_sxfm in data_preproc.items():
    for dxfm, data_preproc_sxfm_dxfm in data_preproc_sxfm.items():
        gr_dat = data_preproc_sxfm_dxfm[np.logical_and(subjects == refsub, sessions==refses),:][0,:].reshape((nv, nv)).flatten()
        for i in range(nv**2):
            data.append([sxfm, dxfm, int(col_ix[i] + 1), int(row_ix[i] + 1), gr_dat[i]])
        for dsi in np.unique(datasets):
            for dsj in np.unique(datasets):
                dsids = np.array([ds in [dsi, dsj] for ds in datasets])
                data_avg.append([sxfm, dxfm, dsi, dsj, data_preproc_sxfm_dxfm[dsids,].mean()])
            
dat_df = pd.DataFrame(data, columns=["Sxfm", "Dxfm", "Column", "Row", "Value"])
dat_avg_df = pd.DataFrame(data_avg, columns=["Sxfm", "Dxfm", "Dataset1", "Dataset2", "Average"])
print(dat_df.head(n=20))
print(dat_avg_df.head(n=20))
dat_df.to_csv('../data/summary/proc_graph.csv')
dat_avg_df.to_csv('../data/summary/avg_gr_weights.csv')

   Sxfm Dxfm  Column  Row     Value
0   raw  raw       1    1  0.000000
1   raw  raw       2    1  0.524848
2   raw  raw       3    1  0.579500
3   raw  raw       4    1  0.381118
4   raw  raw       5    1  0.445685
5   raw  raw       6    1  0.419384
6   raw  raw       7    1  0.658296
7   raw  raw       8    1  0.616606
8   raw  raw       9    1  0.512816
9   raw  raw      10    1  0.618475
10  raw  raw      11    1  0.733692
11  raw  raw      12    1  0.449010
12  raw  raw      13    1  0.674708
13  raw  raw      14    1  0.513455
14  raw  raw      15    1  0.822476
15  raw  raw      16    1  0.623240
16  raw  raw      17    1  0.623368
17  raw  raw      18    1  0.543928
18  raw  raw      19    1  0.358827
19  raw  raw      20    1  0.556902
   Sxfm Dxfm Dataset1 Dataset2   Average
0   raw  raw    BMB_1    BMB_1  0.356484
1   raw  raw    BMB_1     BNU1  0.362395
2   raw  raw    BMB_1     BNU2  0.353954
3   raw  raw    BMB_1     BNU3  0.356904
4   raw  raw    BMB_1     HNU1  0.34417

1458